# Local LLM(Falcon 70B) 및 LangChain을 사용한 개인 챗봇

https://www.mlexpert.io/prompt-engineering/chatbot-with-local-llm-using-langchain

단일 GPU에서 로컬 LLM을 사용하여 ChatGPT와 같은 성능의 비공개 챗봇을 구축할 수 있습니까?

이 자습서에서는 Falcon 70B를 사용합니다1 LangChain을 사용하여 대화 메모리를 유지하는 챗봇을 구축합니다. 단일 T4 GPU를 활용하고 8 비트 (~ 6 토큰 / 초)로 모델을 로드하여 적절한 성능을 얻을 수 있습니다. 또한 다음과 같이 출력 품질과 속도를 개선하는 기술을 살펴봅니다.

중지 기준 : LLM "rambling"의 시작을 감지하고 생성을 중지합니다.
정리 출력 : 때때로 LLM이 이상하거나 추가 토큰을 출력하는 경우 출력에서 토큰을 지우는 방법을 보여 드리겠습니다
채팅 기록 저장: LLM이 대화 기록을 기억할 수 있도록 메모리를 사용합니다.

## 설치

-----

필요한 종속성을 설치하는 것부터 시작하겠습니다.

In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.40.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.30.0 --progress-bar off
!pip install -qqq accelerate==0.21.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off
!pip install -qqq langchain==0.0.233 --progress-bar off

필요한 import 목록은 다음과 같습니다.

In [ ]:
import re
import warnings
from typing import List
 
import torch
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
)
 
warnings.filterwarnings("ignore", category=UserWarning)

## 모델 로드

직접 로컬에 있는 모델을 로드할 수 있습니다.

In [ ]:
import os

cwd = os.getcwd()

MODEL_NAME = cwd + "/models/falcon-7b-instruct"
 
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, trust_remote_code=True, load_in_8bit=True, device_map="auto"
)
model = model.eval()
 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)